<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Предобработка-текста" data-toc-modified-id="Предобработка-текста-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Предобработка текста</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Деление-данных" data-toc-modified-id="Деление-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Деление данных</a></span></li><li><span><a href="#Векторизация-текста" data-toc-modified-id="Векторизация-текста-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Векторизация текста</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>SGDClassifier</a></span></li></ul></li><li><span><a href="#Финальная-модель" data-toc-modified-id="Финальная-модель-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Финальная модель</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import pandas as pd
import numpy as np
import re

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Слава\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Подготовка

In [2]:
try:
    data = pd.read_csv('toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Не знаю, чем вызвано появление первого столбца, но похоже, что это дублированный столбец с индексами.

In [4]:
# Если вдруг этот столбец появится только на моей локальной машине
try:
    data = data.drop(['Unnamed: 0'], axis=1)
except:
    print('Столбца \'Unnamed: 0\' нет')

### Предобработка текста

In [5]:
# Функция будет чистить от мусора текст и приводить к единому виду
def cleaning(text):
    text = re.sub(r'[?(\r|\n)]', ' ', text)
    text = re.sub(r'[^a-zA-Z ]+', '', text)
    text = text.lower()
    text = ' '.join(text.split())
    
    return text

In [6]:
# Просто лемматизируем текст
lemmatizer = WordNetLemmatizer()
def lemmatize(text):    
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    
    return lemmatized_output

In [7]:
%%time
data['text'] = data['text'].apply(cleaning)
data['text'] = data['text'].apply(lemmatize)

corpus = data['text']
corpus

CPU times: total: 57.9 s
Wall time: 58 s


0         explanation why the edits made under my userna...
1         daww he match this background colour im seemin...
2         hey man im really not trying to edit war it ju...
3         more i cant make any real suggestion on improv...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287    and for the second time of asking when your vi...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm there no actual article for prosti...
159290    and it look like it wa actually you who put on...
159291    and i really dont think you understand i came ...
Name: text, Length: 159292, dtype: object

In [8]:
corpus[1]

'daww he match this background colour im seemingly stuck with thanks talk january utc'

## Обучение

### Деление данных

In [9]:
data.shape

(159292, 2)

Данных достаточное количество, думаю имеется возможность их разделить стандартно в пропорциях `3:1`.

In [10]:
features = corpus
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size = .25, random_state = 12345)

features_train.shape[0], features_test.shape[0]

(119469, 39823)

In [11]:
del data, corpus, features, target

### Векторизация текста

In [12]:
count_tf_idf = TfidfVectorizer(stop_words=nltk_stopwords.words('english'))

tfidf_train = count_tf_idf.fit_transform(features_train)
tfidf_test = count_tf_idf.transform(features_test)

In [13]:
del count_tf_idf

### LogisticRegression

In [14]:
%%time
model_lr = LogisticRegression()


param_grid = {'solver':['liblinear'],
              'penalty' : ['l1', 'l2'],
              'C': list(range(1,15,3)),
              'random_state':[12345]
             }

grid_lr = GridSearchCV(model_lr, param_grid=param_grid, scoring='f1', cv=3, verbose=True)
grid_lr.fit(tfidf_train, target_train)

print(grid_lr.best_params_)
print('f1:', grid_lr.best_score_)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
{'C': 4, 'penalty': 'l1', 'random_state': 12345, 'solver': 'liblinear'}
f1: 0.7649612906785448
CPU times: total: 30.2 s
Wall time: 29.7 s


### LGBMClassifier

In [15]:
%%time
model_lgbm = LGBMClassifier()

param_grid = {'boosting_type': ['gbdt', 'dart', 'rf'],
              'learning_rate': [0.1, 0.2, 0.4],
              'max_depth': [5, 10, 15],
              'random_state':[12345]
             }

grid_lgbm = GridSearchCV(model_lgbm, param_grid=param_grid, scoring='f1', cv=3, verbose=True)
grid_lgbm.fit(tfidf_train, target_train)

print(grid_lgbm.best_params_)
print('f1:', grid_lgbm.best_score_)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'boosting_type': 'gbdt', 'learning_rate': 0.4, 'max_depth': 15, 'random_state': 12345}
f1: 0.7490325029423879
CPU times: total: 30min 6s
Wall time: 5min 32s


### SGDClassifier

In [16]:
%%time
model_sgdc = SGDClassifier()
param_grid = [{'loss':['hinge', 'log', 'modified_huber'],
                'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'],
                'eta0':[0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
                'random_state':[12345]
               }]


grid_sgdc = GridSearchCV(model_sgdc, param_grid, scoring='f1', cv=4)
grid_sgdc.fit(tfidf_train, target_train)

print(grid_sgdc.best_params_)
print('f1:', grid_sgdc.best_score_)

{'eta0': 0.1, 'learning_rate': 'constant', 'loss': 'modified_huber', 'random_state': 12345}
f1: 0.7109472246245846
CPU times: total: 1min 30s
Wall time: 1min 29s


## Финальная модель

Лучший результат из трех проверяемых моделей показала `LogisticRegression`. Именно ее мы возьмем как финальную.

In [17]:
final_model = LogisticRegression(**grid_lr.best_params_)
final_model.fit(tfidf_train, target_train)

final_prediction = final_model.predict(tfidf_test)
print(f1_score(target_test, final_prediction))

0.7864526659412405


## Вывод

* Мы подготовили данные для обучения:
 - Очистили и леммматизировали текст
 - Векторизировали данные
* Мы проверили несколько моделей из которых лучшей оказалась `LogisticRegression`. 
* Проверив финальную модель на F1 метрику, мы достигли результата, который требовался.